In [1]:
import pandas as pd
import numpy as np
import catboost
from sklearn.model_selection import StratifiedKFold

/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train = pd.read_csv('../inputs/train.csv')

In [3]:
drop_cols=['target', 'id']
for column in train.columns:
    if('calc' in column):
        drop_cols.append(column)
X = train[[
    'ps_car_13',
	'ps_reg_03',
	'ps_ind_05_cat',
	'ps_ind_03',      
	'ps_ind_15',      
	'ps_reg_02',        
	'ps_car_14',  
	'ps_car_12',       
	'ps_car_01_cat',       
	'ps_car_07_cat',       
	'ps_ind_17_bin',      
	'ps_car_03_cat',    
	'ps_reg_01',        
	'ps_car_15',       
	'ps_ind_01',         
	'ps_ind_16_bin',      
	'ps_ind_07_bin',    
	'ps_car_06_cat',     
	'ps_car_04_cat',   
	'ps_ind_06_bin',      
	'ps_car_09_cat',      
	'ps_car_02_cat',     
	'ps_ind_02_cat',    
	'ps_car_11',          
	'ps_car_05_cat',      
	'ps_calc_09',         
	'ps_calc_05',       
	'ps_ind_08_bin',     
	'ps_car_08_cat',   
	'ps_ind_09_bin',     
	'ps_ind_04_cat',    
	'ps_ind_18_bin',    
	'ps_ind_12_bin',     
	'ps_ind_14'  
]]
y = train['target']

In [4]:
n_folds = 10
skf = StratifiedKFold(n_splits=n_folds)
skf.get_n_splits(X, y)

10

In [5]:
cat_feature_inds = []
for index, column in enumerate(X.columns):
    if('cat' in column):
        cat_feature_inds.append(index)

In [6]:
x_matrix = X.as_matrix()
y_matrix = y.as_matrix()

In [7]:
average_iter = 0
num_iterations = 0
for train_index, test_index in skf.split(x_matrix, y_matrix):
    print('Iteration ' + str(num_iterations))
    num_iterations+=1
    X_train, X_test = x_matrix[train_index], x_matrix[test_index]
    y_train, y_test = y_matrix[train_index], y_matrix[test_index]
    eval_set = X_test, y_test
    model = catboost.CatBoostClassifier(
        depth=11.0, 
        l2_leaf_reg=19.753882117849393, 
        learning_rate=0.11107217807868233, 
        iterations = 1500,
        eval_metric='AUC',
        use_best_model=True,
        od_type='Iter',
        loss_function='Logloss')
    model.fit(X_train, y_train, eval_set=eval_set, cat_features=cat_feature_inds)
    print('Num trees ' + str(model.tree_count_))
    average_iter += model.tree_count_

Iteration 0
Num trees 173
Iteration 1
Num trees 168
Iteration 2
Num trees 133
Iteration 3
Num trees 236
Iteration 4
Num trees 122
Iteration 5
Num trees 165
Iteration 6
Num trees 173
Iteration 7
Num trees 172
Iteration 8
Num trees 178
Iteration 9
Num trees 176


In [8]:
print(average_iter//n_folds)

169
